In [7]:
from sklearn.utils.fixes import loguniform
import numpy as np
import doepy
from doepy import build
import pyDOE2
import pandas as pd

In [42]:
from tqdm import tqdm
from sklearn.model_selection import cross_val_score


def eval_hyper_parameter(clf_class, params, X, y):
    clf = clf_class(**params)
    return np.mean(cross_val_score(clf, X, y))

def eval_plan(clf_class, plan, X, y, key="results"):
    with tqdm(total=len(plan)) as pbar:
        def to_eval(params): 
            res = eval_hyper_parameter(clf_class, params, X, y)
            pbar.update(1)
            return res
        plan[key] = plan.apply(to_eval, axis=1)
    return plan

In [43]:
parameters ={'kernel':["linear", "poly", "rbf", "sigmoid"],
    'C':[float("1e{}".format(i)) for i in range(-5, 5)], 
    'gamma':['scale', 'auto']}
plan = build.full_fact(
    parameters
)

In [44]:
for key in {"gamma", "kernel"}:
    plan[key] = plan[key].apply(lambda elt: parameters[key][int(elt)])

In [45]:
from sklearn.datasets import make_classification
X, y = make_classification()

In [46]:
from sklearn.svm import SVC
eval_plan(SVC, plan, X, y)

100%|██████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 142.34it/s]


,kernel,C,gamma,results
0,linear,0.00001,scale,0.89
1,poly,0.00001,scale,0.56
2,rbf,0.00001,scale,0.90
3,sigmoid,0.00001,scale,0.89
4,linear,0.00010,scale,0.89
...,...,...,...,...
75,sigmoid,1000.00000,auto,0.90
76,linear,10000.00000,auto,0.90
77,poly,10000.00000,auto,0.74
78,rbf,10000.00000,auto,0.90


In [11]:

{'C': loguniform(1e0, 1e3),
 'gamma': loguniform(1e-4, 1e-3),
 'kernel': ['rbf'],
 'class_weight':['balanced', None]}

{'C': <scipy.stats._distn_infrastructure.rv_frozen at 0x269c565b448>,
 'gamma': <scipy.stats._distn_infrastructure.rv_frozen at 0x269c566d048>,
 'kernel': ['rbf'],
 'class_weight': ['balanced', None]}

In [12]:
build.space_filling_lhs(
{'Pressure':[40,55,70],
'Temperature':[290, 320, 350],
'Flow rate':[0.2,0.4], 
'Time':[5,8],},
num_samples = 12
)

Pressure had more than two levels. Assigning the end point to the high level.
Temperature had more than two levels. Assigning the end point to the high level.


,Pressure,Temperature,Flow rate,Time
0,70.000000,290.000000,0.254545,6.090909
1,67.272727,328.181818,0.290909,7.454545
2,59.090909,350.000000,0.218182,7.727273
3,42.727273,344.545455,0.272727,5.000000
4,56.363636,322.727273,0.400000,6.363636
5,40.000000,300.909091,0.236364,7.181818
6,45.454545,306.363636,0.200000,5.272727
7,64.545455,317.272727,0.345455,5.818182
8,50.909091,295.454545,0.309091,8.000000
9,48.181818,311.818182,0.363636,5.545455
